# Capstone Project

## Purpose/Scope
This project will be to import, clean, and link supplied data sets from Udacity for further examination by Data Scientists for analytical information regarding immigration as it relates to  airport/port of arrival city with additional data on the demographic data of the port of arrival. The primary use case for this will be to analyze immigration patterns into certain ports of entry as they relate to the country of residence and whether or not this is a personal, school, or business reason for visiting the US. 

Spark will be the analysis tool of choice due to it's ability to handle Big Data (many millions+) of rows well, especially if deployed to a cluster environment. 

## Data

Datasets were provided by Udacity from other open source repositories. 
* Airport Code Table - Table of Airport Codes and the corresponding city & basic city metadata - Source: [Our Airports](http://ourairports.com/data/) via [DataHub](https://datahub.io/core/airport-codes#data)
* World Temperature Data - Data back to the 1700's for many world cities. - Source: [Berkeley Earth](http://berkeleyearth.org/) via [Kaggle](https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data)
* US City Demographic Data - Demographics/Population data on most US Cities and CDP's from [US Census Bureau](https://www.census.gov/data/developers/about/terms-of-service.html) via [OpenDataSoft](https://public.opendatasoft.com/explore/dataset/us-cities-demographics/table/?dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6InVzLWNpdGllcy1kZW1vZ3JhcGhpY3MiLCJvcHRpb25zIjp7fX0sImNoYXJ0cyI6W3siYWxpZ25Nb250aCI6dHJ1ZSwidHlwZSI6ImNvbHVtbiIsImZ1bmMiOiJBVkciLCJ5QXhpcyI6Im1lZGlhbl9hZ2UiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiIjRkY1MTVBIn1dLCJ4QXhpcyI6ImNpdHkiLCJtYXhwb2ludHMiOjUwLCJzb3J0IjoiIn1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D)
* I94 Immigration Data - Data on Immigration/Visitation in April 2016 from the Department of Commerce. Includes some demographic data on visitor. Original data set no longer available online, but provided by Udacity as an archive in Parquet format. - Current program site source: [I91 Arrivals Program](https://www.trade.gov/i-94-arrivals-program)


## Setup
Import requirements

In [116]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, DecimalType, TimestampType
from datetime import date, datetime, timedelta
from os.path import exists

Set output location / AWS Credentials (alternatively, set in a config file). For the purposes of testing and cost savings, files are loaded in current state to a my_local_data folder, but changing the comments will result in an output to an Amazon S3 bucket.

In [2]:
output_location = 'my_local_data/'
#output_location = 's3a://AWSBUCKETNAME/AWSKEYNAME'
#os.environ["AWS_ACCESS_KEY_ID"]= 'YOURKEY'
#os.environ["AWS_SECRET_ACCESS_KEY"] = 'YOURSECRET'

Build initial spark session

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

* Setup Date Function for Converting SAS Date formats to Regular Date format (SAS days are # of days since beginning of 1960)
* Quality Check Function For confirming table information and column information
* Name function to return dataframe name when passed as an argument into another function

In [4]:
@udf(TimestampType())
def sas_date (mydate):
    if mydate is None:
        z = None
    else: 
        z = datetime(1960,1,1) + timedelta(days=int(mydate))
    return z

In [5]:
#This function provided on Stack Overflow to get a dataframe name when the object itself is passed
# as an argument. Written/provided by cors https://stackoverflow.com/a/54138398

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name


In [6]:
def qualityCheck(a):
    # Function takes a list of lists as a parameter, with the first element of the
    # sublist being a table DF and all other values (unlimited) being column/field names quoted.
    # Example: [[table_name,"primaryKeyName","secondFieldName"], [table_name2,"columnA"]]
    # Quality Check will 
    #   A) Verify the DF table has 1 or more records
    #   B) Verify each passed field/column has no null records
    
    errors = []
    for i in range(len(a)):
        table = a[i][0]
        tableName = get_df_name(table)
        print ("Checking table name {}".format( tableName ))
        cnt = table.count()
        if not cnt >= 1:
            errors.append("{} FAILED. Expected more than 0 records. Received {} ".format(tableName, cnt))
            print("  FAILED! {} expected more than 0 records. Received {} ".format(tableName, cnt))
        else:
            print ("  SUCCESS! Found {} records.".format(cnt))
        for j in range(1, len(a[i])):
            columnName = a[i][j]
            print ("  Checking column name {} for NULL values".format( columnName ))
            cnt = table.where(col(columnName).isNull()).count()
            if cnt > 0:
                errors.append("{}.{} Column FAILED. Expected 0 NULL records. Received {} ".format(tableName, columnName, cnt))
                print("     FAILED! {} NULL Found.".format(cnt))
            else:
                print("    SUCCESS! 0 NULL Found.")
    if len(errors) > 0:
        raise ValueError("Errors found in data quality check.\n{}".format('\n'.join(map(str, errors))))

## Extract Data / Read In Data to Spark

Import i94 data, run QC function, and preview

In [7]:
i94data =spark.read.parquet("./sas_data")

In [8]:
i94data_revised = i94data.withColumn("departure_date",sas_date(col("depdate")))
i94data_revised = i94data_revised.withColumn("arrival_date",sas_date(col("arrdate")))

In [9]:
tableDataToCheck = [[i94data_revised, "cicid", "arrival_date","admnum"]]
qualityCheck(tableDataToCheck)

Checking table name i94data_revised
  SUCCESS! Found 3096313 records.
  Checking column name cicid for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name arrival_date for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name admnum for NULL values
    SUCCESS! 0 NULL Found.


In [10]:
i94data_revised.limit(5).toPandas().head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype,departure_date,arrival_date
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,...,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1,2016-05-08,2016-04-30
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,...,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1,2016-05-17,2016-04-30
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,...,1987.0,10292016,M,None,DL,9.495641e+10,00040,B1,2016-05-08,2016-04-30
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,...,1987.0,10292016,F,None,DL,9.495645e+10,00040,B1,2016-05-14,2016-04-30
4,5748521.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,...,1988.0,10292016,M,None,DL,9.495639e+10,00040,B1,2016-05-14,2016-04-30


Import City Demographics, filter out race information creating duplicate records (not needed for current scope), run QC function, and preview.

In [11]:
us_cities=spark.read.csv("./us-cities-demographics.csv", sep=';', header=True)

In [12]:
us_cities_revised = us_cities.select('City',
 'State',
 'Median Age',
 'Male Population',
 'Female Population',
 'Total Population',
 'Number of Veterans',
 'Foreign-born',
 'Average Household Size',
 'State Code').dropDuplicates().orderBy("state","city")

In [13]:
tableDataToCheck = [[us_cities_revised, "City", "State Code"]]
qualityCheck(tableDataToCheck)

Checking table name us_cities_revised
  SUCCESS! Found 596 records.
  Checking column name City for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name State Code for NULL values
    SUCCESS! 0 NULL Found.


In [14]:
us_cities_revised.limit(5).toPandas().head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code
0,Birmingham,Alabama,35.6,102122,112789,214911,13212,8258,2.21,AL
1,Dothan,Alabama,38.9,32172,35364,67536,6334,1699,2.59,AL
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL
3,Huntsville,Alabama,38.1,91764,97350,189114,16637,12691,2.18,AL
4,Mobile,Alabama,38.0,91275,103030,194305,11939,7234,2.4,AL


Import Airport data, fix state column (from US-AL to AL), remove irregular types (immigration doesn't happen by balloon), and specify only airports with an iata_code (INS immigration does not traditionally happen at an airport small enough to not be iata coded). Finally, preview.

In [15]:
us_airports=spark.read.csv("./airport-codes_csv.csv", sep=',', header=True)

In [16]:
us_airports_revised = us_airports.withColumn('state', regexp_replace("iso_region","US-","")).\
                drop('iso_region').filter(us_airports.type != "small_airport").\
                filter(us_airports.type != "heliport").\
                filter(us_airports.type != "closed").\
                filter(us_airports.type != "balloonport").\
                filter(us_airports.type != "seaplane_base").\
                filter(us_airports.iso_country == "US").\
                filter(us_airports.iata_code.isNotNull())

In [17]:
tableDataToCheck = [[us_airports_revised, "ident", "iata_code","municipality","state"]]
qualityCheck(tableDataToCheck)

Checking table name us_airports_revised
  SUCCESS! Found 820 records.
  Checking column name ident for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name iata_code for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name municipality for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name state for NULL values
    SUCCESS! 0 NULL Found.


In [18]:
us_airports_revised.limit(5).toPandas().head()

,ident,type,name,elevation_ft,continent,iso_country,municipality,gps_code,iata_code,local_code,coordinates,state
0,5A8,medium_airport,Aleknagik / New Airport,66,NA,US,Aleknagik,5A8,WKK,5A8,"-158.617996216, 59.2826004028",AK
1,KABE,medium_airport,Lehigh Valley International Airport,393,NA,US,Allentown,KABE,ABE,ABE,"-75.44080352783203, 40.652099609375",PA
2,KABI,medium_airport,Abilene Regional Airport,1791,NA,US,Abilene,KABI,ABI,ABI,"-99.68190002440001, 32.4113006592",TX
3,KABQ,large_airport,Albuquerque International Sunport,5355,NA,US,Albuquerque,KABQ,ABQ,ABQ,"-106.609001, 35.040199",NM
4,KABR,medium_airport,Aberdeen Regional Airport,1302,NA,US,Aberdeen,KABR,ABR,ABR,"-98.42179870605469, 45.449100494384766",SD


Import SAS Data Label file. 

The source of this function is Udacity Mentor  Anshul R on mentor help site https://knowledge.udacity.com/questions/125439 for Udacity student use.
Decoding this file's irregular format was not in scope of the course, so I used the code provided with minimal modifications to bring the data into a proper Spark DF.

Run quality checks on each, then preview new dataframes.

In [19]:


schema = StructType([ \
    StructField("ID",StringType(),True), \
    StructField("Name",StringType(),True), \
  ])

with open('./I94_SAS_Labels_Descriptions.SAS') as f:
    f_content = f.read()
    f_content = f_content.replace('\t', '')
def code_mapper(file, idx):
    f_content2 = f_content[f_content.index(idx):]
    f_content2 = f_content2[:f_content2.index(';')].split('\n')
    f_content2 = [i.replace("'", "") for i in f_content2]
    dic = [i.split('=') for i in f_content2[1:]]
    dic = dict([str(i[0].strip()), i[1].strip()] for i in dic if len(i) == 2)
    return dic
i94cit_res = spark.createDataFrame(code_mapper(f_content, "i94cntyl").items(), schema)
i94port = spark.createDataFrame(code_mapper(f_content, "i94prtl").items(), schema)
i94mode = spark.createDataFrame(code_mapper(f_content, "i94model").items(), schema)
i94addr = spark.createDataFrame(code_mapper(f_content, "i94addrl").items(), schema)
i94mode = spark.createDataFrame(code_mapper(f_content, "i94model").items(), schema)
i94visa = spark.createDataFrame(list(({'1':'Business',
'2': 'Pleasure',
'3' : 'Student'}).items()), schema)

Because the i94 table does not follow IATA naming, we will utilize the Port Table (derived from the SAS Label file above) to parse City and State. This can then be mapped with semi-resonable accuracy to the airport table if also filtering on the mode table to be air travel only. Remove entries that do not have a state code parseable.

In [20]:
i94port_revised = i94port.withColumn('City',split(i94port.Name,', ').getItem(0)).\
                                    withColumn('State',split(i94port.Name,', ').getItem(1)).\
                                    drop("Name")
i94port_revised = i94port_revised.filter(i94port_revised.State.isNotNull())

In [21]:
tableDataToCheck = [[i94port_revised, "ID", "City", "State"]]
qualityCheck(tableDataToCheck)

Checking table name i94port_revised
  SUCCESS! Found 582 records.
  Checking column name ID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name City for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name State for NULL values
    SUCCESS! 0 NULL Found.


In [22]:
tableDataToCheck = [[i94cit_res, "ID", "Name"]]
qualityCheck(tableDataToCheck)

Checking table name i94cit_res
  SUCCESS! Found 289 records.
  Checking column name ID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.


In [23]:
i94port_revised.limit(5).toPandas().head()

,ID,City,State
0,ALC,ALCAN,AK
1,ANC,ANCHORAGE,AK
2,BAR,BAKER AAF - BAKER ISLAND,AK
3,DAC,DALTONS CACHE,AK
4,PIZ,DEW STATION PT LAY DEW,AK


In [24]:
i94cit_res.limit(5).toPandas().head()

,ID,Name
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [25]:
tableDataToCheck = [[i94mode, "ID", "Name"]]
qualityCheck(tableDataToCheck)

Checking table name i94mode
  SUCCESS! Found 4 records.
  Checking column name ID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.


In [26]:
i94mode.limit(5).toPandas().head()

,ID,Name
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [27]:
tableDataToCheck = [[i94addr, "ID", "Name"]]
qualityCheck(tableDataToCheck)

Checking table name i94addr
  SUCCESS! Found 55 records.
  Checking column name ID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.


In [28]:
i94addr.limit(5).toPandas().head()

,ID,Name
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [29]:
tableDataToCheck = [[i94visa, "ID", "Name"]]
qualityCheck(tableDataToCheck)

Checking table name i94visa
  SUCCESS! Found 3 records.
  Checking column name ID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.


In [30]:
i94visa.limit(5).toPandas().head()

,ID,Name
0,1,Business
1,2,Pleasure
2,3,Student


In [31]:
tableDataToCheck = [[i94mode, "ID", "Name"]]
qualityCheck(tableDataToCheck)

Checking table name i94mode
  SUCCESS! Found 4 records.
  Checking column name ID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.


In [32]:
i94mode.limit(5).toPandas().head()

,ID,Name
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


## Data Model

### Concept

The model is made up by the previous defined dataframes:

|Name|Data Type|Description|Assumptions|
|---|----|---|----|
|i94data|Fact Data|Travel/Immigration Data from USNTTO|cicid is a unique identifer per person|
|i94visa|Dimension Data|Visa Type|

|i94port|Dimension Data|Port of Entry|
|i94mode|Dimension Data|Type of Travel Entry|
|i94cit_res|Dimension Data|Origin/Country of Residence|
|us_airports|Dimension Data|Listing of US Airpoprts|
|us_cities|Dimension Data|Listing of US Cities|

<!--|us_temp|Fact Data|Temperature by Month by City|-->
<!--|i94addr|Dimension Data|State/Location of Destination|-->

Data will be loaded from these staging df's into a proper star schema. Based on the data set having only one two updating table (immigration and less frequently updating, person) The primary fact table will be made up of actual immigrations, with dimensional tables for airport, time, country, mode of transportation, i94 port names, and visa type.

<img src="dbSchema.png" size = '30%'>
<i><div style="text-align: right">  Graphic built with https://dbdiagram.io/d </div></i>

In a traditional ETL, we would perhaps be joining new data on some regular basis, requiring modification of the below ETL process ti import only new data. As it stands, this is a static data set, so a single insert is implied.


### Transform / Define Tables

In [33]:

person_table = i94data_revised.selectExpr("cicid as personID", "i94res as country", \
                                  "i94bir as age", "gender","admnum as admissionId", "occup as occupation")

In [34]:
time_table = i94data_revised.selectExpr("arrival_date as arrivalDate", "year(arrival_date) as year", \
                                        "month(arrival_date) as month", "day(arrival_date) day", \
                                        "hour(arrival_date) as hour", "weekofyear(arrival_date) week", \
                                        "dayofweek(arrival_date) weekDay").dropDuplicates()

In [35]:
country_table = i94cit_res.selectExpr("ID as countryID", "Name")

In [36]:
airport_table = us_airports_revised.selectExpr("ident as airportID", "name as name", \
                                   "municipality as city", "state", "iata_code as iataCode")

In [37]:
visa_table = i94visa.selectExpr("ID as visaID", "Name")

In [38]:
port_table = i94port_revised.selectExpr("ID as portID", "City", "State")

In [39]:
mode_table = i94mode.selectExpr("ID as modeID", "Name")

In [40]:
immigration_fact_table = i94data_revised.selectExpr("cicid as personID", "departure_date", "arrival_date", \
                           "i94visa as visaTypeID", "i94port as portID", \
                           "airline as airlineName", "fltno as flightNumber", "admnum as admissionNumber",
                           "i94mode as modeID")

## Load / Quality Checks

### Parquet Export/Imports
Define tables to upload in spark/parquet. Count records going out, then read data back in and do another count to ensure everything is good.

In [49]:
tables = [immigration_fact_table, port_table, time_table, \
          mode_table, country_table, visa_table, \
          airport_table, person_table]


In [51]:

for table in tables:
        fullloc = output_location + get_df_name(table)
        table.write.mode("overwrite").parquet(fullloc)
        print("Exporting {} records to {}".format(table.count(),fullloc))
    

Exporting 3096313 records from my_local_data/immigration_fact_table
Exporting 582 records from my_local_data/port_table
Exporting 30 records from my_local_data/time_table
Exporting 4 records from my_local_data/mode_table
Exporting 289 records from my_local_data/country_table
Exporting 3 records from my_local_data/visa_table
Exporting 820 records from my_local_data/airport_table
Exporting 3096313 records from my_local_data/person_table


In [56]:
for table in tables:
        fullloc = output_location + get_df_name(table)
        table = spark.read.parquet(fullloc)
        print("{} imported {} records".format(fullloc, table.count()))
    

my_local_data/immigration_fact_table imported. 3096313 records found
my_local_data/port_table imported. 582 records found
my_local_data/time_table imported. 30 records found
my_local_data/mode_table imported. 4 records found
my_local_data/country_table imported. 289 records found
my_local_data/visa_table imported. 3 records found
my_local_data/airport_table imported. 820 records found
my_local_data/person_table imported. 3096313 records found


### Data Quality Checks

In [58]:
tableDataToCheck = [
    [port_table, "portID", "Name"],
    [mode_table, "modeID", "Name"],
    [country_table, "countryID", "Name"],
    [visa_table, "visaID", "Name"],
    [airport_table, "airportID", "City", "State"],
    [time_table, "arrival_date"],
    [person_table, "personID"],
    [immigration_fact_table, "personID", "arrival_date","portID", "visaTypeID", "arrival_date"]
]
qualityCheck(tableDataToCheck)

Checking table name port_table
  SUCCESS! Found 582 records.
  Checking column name portID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.
Checking table name mode_table
  SUCCESS! Found 4 records.
  Checking column name modeID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.
Checking table name country_table
  SUCCESS! Found 289 records.
  Checking column name countryID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.
Checking table name visa_table
  SUCCESS! Found 3 records.
  Checking column name visaID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name Name for NULL values
    SUCCESS! 0 NULL Found.
Checking table name airport_table
  SUCCESS! Found 820 records.
  Checking column name airportID for NULL values
    SUCCESS! 0 NULL Found.
  Checking column name City for NULL v

### Create Data Dictionary
This is a one time process to create dictionary. It should be filled out manually.

In [127]:
from IPython.display import Markdown, display
if not exists("dictionary.md"):
    f = open("dictionary.md", "x")
    f.write("# Data Dictionary\n\n")
    for table in tables:
        f.write("##" + get_df_name(table) + '\n\n')
        f.write("|Name|Type|Description|\n")
        f.write("|---|---|---|\n")
        for field in table.schema.fields:
            f.write("|" + field.name + "|" + str(field.dataType) + "||\n")
        f.write ("\n")
    f.close()
    print ("Dictionary must be filled out with proper field descriptions! Open dictionary.md to continue.")
    
else:
    print ("Dictionary file already exists.")
    display(Markdown("dictionary.md"))
    




Dictionary file already exists.


# Data Dictionary
## immigration_fact_table

|Name|Type|Description|
|---|---|---|
|personID|DoubleType|Person Table Reference ID|
|departure_date|TimestampType|Date of Departure for Visitor|
|arrival_date|TimestampType|Date of Arrival for Visitor|
|visaTypeID|DoubleType|Visa Table Reference ID|
|portID|StringType|Port Table Reference ID|
|airlineName|StringType|Airline Code Name|
|flightNumber|StringType|Airline Flight Number|
|admissionNumber|DoubleType|Unique key for immigration entry|
|modeID|DoubleType|Mode Table Reference ID|

## port_table

|Name|Type|Description|
|---|---|---|
|portID|StringType|Primary Key Identifier|
|City|StringType|Port of Entry City|
|State|StringType|Port of Entry State|

## time_table

|Name|Type|Description|
|---|---|---|
|arrivalDate|TimestampType|Primary Key Identifier|
|year|IntegerType|Arrival Year|
|month|IntegerType|Arrival Month|
|day|IntegerType|Arrival Day|
|hour|IntegerType|Arrival Hour|
|week|IntegerType|Arrival Week|
|weekDay|IntegerType|Arrival DayofWeek (1-7 Sun-Sat)|

## mode_table

|Name|Type|Description|
|---|---|---|
|modeID|StringType|Primary Key Identifier|
|Name|StringType|Mode of Transportation for Entry|

## country_table

|Name|Type|Description|
|---|---|---|
|countryID|StringType|Primary Key Identifier|
|Name|StringType|Country of Residence|

## visa_table

|Name|Type|Description|
|---|---|---|
|visaID|StringType|Primary Key Identifier|
|Name|StringType|Visa Type/Reason for Visit|

## airport_table

|Name|Type|Description|
|---|---|---|
|airportID|StringType|Primary Key Identifier|
|name|StringType|Airport Name|
|city|StringType|Airport City|
|state|StringType|Airport State|
|iataCode|StringType|Airport IATA Code|

## person_table

|Name|Type|Description|
|---|---|---|
|personID|DoubleType|Primary Key Identifier|
|country|DoubleType|Country Table Reference ID|
|age|DoubleType|Person Age|
|gender|StringType|Person Gender|
|admissionId|DoubleType|Person Admission ID|
|occupation|StringType|Person Occupation|

</align>

## Conclusion

The project required a definition and ETL of data of 1 million+ rows. Data was imported, scrubbed for known bad types of data, transformed to deal with incomplete/poor naming, brought into new data frames, and finally uploaded to the output location (local or AWS). From here, the files can be read back into spark and joined or queried as necessary. A common query would be to join Airport Data, Port Data, Immigration, and Person tables to get proper names of airports the visitors

As this was a large data set, spark was a clear choice for dealing with a large number of records. Because of the parquet files, this could also be spun up into multiple instances easily on AWS clusters.

This data should only be run once in it's current form due to the static nature of the data sources. However, if this were a regular update, once a month would likely suffice for new immigration data.

### Scenarios
* Data Increased by 100x? As this is a Spark Job, I do not forsee any issues based on this being an ETL process that uses Spark. The longest piece is the read and write of parquet data, which only takes a few minutes at the current step. Multiplying this by 100 would require the job to run for several hours, but this could likely be broken down into two or more ETL's for simultaneous runs
* Pipline run at 7AM? This ETL only takes a few minutes to run, so this would not be a challenge as currently written. The only changes necessary would be accounting for new file names and potentially exporting files with a date component in the parquet name. This should be a scheduled job and not run manually. A prefernce would either be a scheduling engine application (such as crontab) or running in a workflow style ETL program like Apache Airflow.
* 100+ People need access to data? Additional EMR clusters would need to be spun up to handle the additionial load, but no other changes would need to be made.